In [2]:
import docx
import random
import math
import numpy as np
import import_ipynb
import MCQGenFile as MCQ
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [5]:
def MakeStarterPack(DocTitle,TopicArr,Pages):
    document = Document()
    
    PuzzleArr = RandomPuzzle((len(TopicArr)-7)*Pages)
    PuzzleQs = PuzzleArr[0]
    PuzzleAns = PuzzleArr[1]
    MatchingQuestionArr = MakeQuestionArray(TopicArr,Pages,5,True)
    MatchingQuestions = MatchingQuestionArr[0]
    MatchingAnswers = MatchingQuestionArr[1]
    OutMatchingAnswers = MatchingAnswers #Matching Answers are eaten up when questions are made, so this is a record of them in correct order.
    QuestionArr = MakeQuestionArray(TopicArr,Pages,15,False)
    Questions = QuestionArr[0]
    Answers = QuestionArr[1]
    TopicNo = 7
    TopicNoCounter = 0
    
    #Creating each page
    while len(Questions) > 0:
        print(f"Question array has a length of {len(Questions)}. Matching Qarray has {len(MatchingQuestions)}, AArray {len(MatchingAnswers)}")
        document.add_heading("Starter with new content: " + TopicArr[TopicNo], 1)
        TopicNoCounter += 1
        if TopicNoCounter % Pages == 0:
            TopicNo += 1
            TopicNoCounter = 0
        document.add_paragraph("Matching: Draw a line to match the questions on the left with answers on the right.")
        MakeMatching(MatchingQuestions[:5],MatchingAnswers[:5],document)
        MatchingQuestions = MatchingQuestions[5:]
        MatchingAnswers = MatchingAnswers[5:]
        document.add_paragraph("Starter Questions: Complete all questions. Remember to check your answers after completing. Go!")
        MakeQuestionGrid(5,3,Questions[:15],document)
        document.add_paragraph("Depth Task: Complete this once you have completed the matching and starter questions.")
        MakePuzzle(PuzzleQs[0],document)
        #Set new puzz array
        document.add_page_break()
        Questions = Questions[15:]
        PuzzleQs = PuzzleQs[1:]
    
    #Answers at end of doc
    document.add_paragraph("Answers")
    print(f"Answers should iterate {round(len(Answers)/15)} times. MatchingAnswers {round(len(MatchingAnswers)/5)} times. PuzzleAns {len(PuzzleAns)} times.")
    for x in range (round(len(Answers)/15)):
        document.add_paragraph(f"Page {x}")
        AnswerStr = ""
        for x in range(5):
            AnswerStr += "\n" + MCQ.prettify(str(OutMatchingAnswers[0]))
            OutMatchingAnswers = OutMatchingAnswers[1:]
        for x in range(15):
            AnswerStr += "\n" + f"{x+1})" + MCQ.prettify(str(Answers[0]))
            Answers = Answers[1:]
        AnswerStr += "\n" + MCQ.prettify(str(PuzzleAns[0]))
        PuzzleAns = PuzzleAns[1:]
        document.add_paragraph(AnswerStr)
        
    
    footer = document.sections[0].footer
    footerpara = footer.paragraphs[0]
    footerpara.text = "Copyright 2022 Maths Plus Mind"
    document.save(DocTitle + '.docx')
    print("Document "+ DocTitle + ".docx has been created.")

def MakeQuestionArray(TopicArray,Pages,QuestionsPerPage,bIsMatching):
    #Will always treat the 9th as the first topic, prior ones are prior knowledge.
    if QuestionsPerPage != 15 and QuestionsPerPage != 5:
        print("Error: In MakeQuestionArray, QuestionsPerPage should be 15 (or 5). Revise/rebuild if this is being changed.")
    OutQuestions = [[],[]]
    TopicCounter = 7
    for Topic in TopicArray[7:]:
        for page in range(Pages):
            #Make questions for matching activity.
            if bIsMatching:
                Questions = globals()["QA_"+TopicArray[TopicCounter-7]](5,False)
                OutQuestions[0] = OutQuestions[0] + Questions[0]
                OutQuestions[1] = OutQuestions[1] + Questions[1]
            
            else:
                #Make Row 1 (past 5-6 topics, focusing on past 5th topic moreso.)
                Questions = globals()["QA_"+TopicArray[TopicCounter-6]](1,False)
                OutQuestions[0] = OutQuestions[0] + Questions[0]
                OutQuestions[1] = OutQuestions[1] + Questions[1]
                Questions = globals()["QA_"+TopicArray[TopicCounter-5]](2,False)
                OutQuestions[0] = OutQuestions[0] + Questions[0]
                OutQuestions[1] = OutQuestions[1] + Questions[1]

                #Make Row 2+3 (current topic with ramp)
                Questions = globals()["QA_"+Topic](6,True)
                OutQuestions[0] = OutQuestions[0] + Questions[0]
                OutQuestions[1] = OutQuestions[1] + Questions[1]

                #Make Row 4+5 (Past 4 topics)
                Questions = globals()["QA_"+TopicArray[TopicCounter-1]](2,False)
                OutQuestions[0] = OutQuestions[0] + Questions[0]
                OutQuestions[1] = OutQuestions[1] + Questions[1]
                Questions = globals()["QA_"+TopicArray[TopicCounter-2]](2,False)
                OutQuestions[0] = OutQuestions[0] + Questions[0]
                OutQuestions[1] = OutQuestions[1] + Questions[1]
                Questions = globals()["QA_"+TopicArray[TopicCounter-3]](1,False)
                OutQuestions[0] = OutQuestions[0] + Questions[0]
                OutQuestions[1] = OutQuestions[1] + Questions[1]
                Questions = globals()["QA_"+TopicArray[TopicCounter-4]](1,False)
                OutQuestions[0] = OutQuestions[0] + Questions[0]
                OutQuestions[1] = OutQuestions[1] + Questions[1]
        TopicCounter += 1
        print(f"MakeQuestionArray: Successful iteration for {Topic}. In total, {len(OutQuestions[0])} questions have been made.")
    
    return OutQuestions

def MakeMatching(qarr,aarr,document):
    r = len(qarr)
    table = document.add_table(rows=r,cols=2)
    table.style = 'Normal Table'
    random.shuffle(aarr)
    for x in range(r):
        qcell = table.rows[x].cells[0]
        qcell_paragraph = qcell.paragraphs[0]
        qcell_paragraph.text = MCQ.prettify(str(qarr[x]))
        acell = table.rows[x].cells[1]
        acell_paragraph = acell.paragraphs[0]
        acell_paragraph.text = MCQ.prettify(str(aarr[x]))

def MakeQuestionGrid(r,c,qarr,document):
    if len(qarr) != r*c:
        print(f"Error: in MakeQuestionGrid, number of questions passed ({len(qarr)}) is not equal to rows x cols ({r*c}) in table.")
        print(f"{qarr}")
    table = document.add_table(rows=r,cols=c)
    table.style = 'Table Grid'
    for x in range(r):
        for y in range(c):
            cell = table.rows[x].cells[y]
            cell_paragraph = cell.paragraphs[0]
            cell_paragraph.text = f"{1+y+c*x}) " + MCQ.prettify(str(qarr[y+c*x]))
            cell.add_paragraph(text=" ",style=None)
            cell_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            cell.add_paragraph(text = " ",style=None)
            cell.add_paragraph(text = " ",style=None)

#Accepts a single puzzle question, entered as an array or single string.
def MakePuzzle(PuzzleArr, document):
    ParagraphStr = ""
    if type(PuzzleArr)==str:
        PuzzleArr = [PuzzleArr] #Making sure that single string questions are treated as lists.
    
    for Sentence in PuzzleArr:
        ParagraphStr += "\n" + MCQ.prettify(Sentence)
        
    document.add_paragraph(text=ParagraphStr,style=None)

In [4]:
#para = document.add_paragraph('   2')
#para.add_run('\nx 3').underline = True
#make font size bigger too!
#document.save('Trial5.docx')

In [4]:
#Accepts two factors and generates a question. Returns the question & answer.
#Note for division questions, b will always be masked.
# 3 UNCOMMON  a * __ = c
# 3 UNCOMMON  __ * a = c
# 5 COMMON   c / a =
# 1 RARE  c / __ = a
# 8 V. COMMON a * b =

def MakeAddQuestion(a,b):
    c = a+b
    chance = random.randint(1,20)
    if chance == 1:
        return [f'{c} - {b} =',f'{a}']
    elif chance <= 4:
        return [f'{a} + ___ = {c}',f'{b}']
    elif chance <= 7:
        return [f'___ + {a} = {c}',f'{b}']
    elif chance <= 12:
        return [f'{c} - {a} =',f'{b}']
    elif random.choice([True,False]):
        return [f'{a} + {b} =',f'{c}']
    else:
        return [f'{b} + {a} =',f'{c}']

def MakeMultQuestion(a,b):
    c = a*b
    chance = random.randint(1,20)
    if chance == 1:
        return [f'{c} ÷ {a} =',f'{b}']
    elif chance <= 4:
        return [f'{a} × ___ = {c}',f'{b}']
    elif chance <= 7:
        return [f'___ × {a} = {c}',f'{b}']
    elif chance <= 12:
        return [f'{c} ÷ {a} =',f'{b}']
    elif random.choice([True,False]):
        return [f'{a} × {b} =',f'{c}']
    else:
        return [f'{b} × {a} =',f'{c}']

In [5]:
#Topics in starter pack to appear in this order
#When you need to retrieve a topic from its number, just use Topics[num-1]
#If this is too messy, you could change each to a list e.g. [Times2,Questions where you need to times by 2.]
ITopics = ["Filler", "PKValueOf", "PKMoney","PKTimesTablesTo6", "PK24hrTime", "PKHoursToMins",
          "Times2", "TimesHalf", "TimesMult5", "TimesTo15", "TimesPowerOfTen",
          "LongMultTo20", "LongMult2dig3dig",
          "ColumnAddSub", "ListFactors", "PrimeNumbers",
          "RoundWhole", "RoundDecimal", 
          "FindMultiplesOf", "MissingNumberInSequence",
         "SquareNumbers", "CubeNumbers"]

#Functions accept a 'ramp' boolean which guides the first three questions with easy intelligent practice, if set to True.
#Functions return [Question, Answer]

def QA_Filler(NumOfQs, bRamp):
    OutQs = [[],[]]
    while NumOfQs > 0:
        OutQs[0].append("Filler")
        OutQs[1].append("Filler")
        NumOfQs -= 1
    return OutQs

#Times tables up to 6, very static question.
def QA_PKTimesTablesTo6(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    while NumOfQs > 0:
        a = random.randint(1,6)
        b = random.randint(1,6)
        OutQs[0].append(f"{a} × {b} =")
        OutQs[1].append(a*b)
        NumOfQs -= 1
    
    return OutQs

def QA_PK24hrTime(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    while NumOfQs > 0:
        a = random.randint(0,23)
        b = random.choice([random.randint(0,60),random.randint(0,6)*10,30*random.randint(0,2)])
        if b == 60:
            b = 0
            a += 1
        if b < 10:
            b = f"0{b}"
        if a >= 13:
            time12 = f"{a-12}:{b}PM"
        else:
            time12 = f"{a}:{b}AM"
        if a < 10:
            time24 = f"0{a}:{b}"
        else:
            time24 = f"{a}:{b}"
        if random.choice([True,False]):
            OutQs[0].append(f"Convert {time12} to 24-hour time.")
            OutQs[1].append(time24)
        else:
            OutQs[0].append(f"Convert {time24} to 12-hour time.")
            OutQs[1].append(time12)
        NumOfQs -= 1
    
    return OutQs

def QA_PKHoursToMins(NumOfQs,bRamp):
    OutQs = [[],[]]
    
    while NumOfQs > 0:
        
        if random.choice([True,False]):
            hours = random.randint(1,5)
            mins = random.randint(0,60)
            OutQs[0].append(f"Convert {hours} hours and {mins} minutes to minutes.")
            OutQs[1].append(hours*60+mins)
        else:
            mins = random.randint(60,180)
            OutQs[0].append(f"Convert {mins} minutes to hours and minutes.")
            OutQs[1].append(f"{math.floor(mins/60)} hours and {mins%60} minutes.")
        NumOfQs -= 1
    return OutQs


#Add money together
def QA_PKMoney(NumOfQs, bRamp):
    OutQs = [[],[]]

    if NumOfQs >= 3 and bRamp == True:
        a = random.randint(1,9)/10 + random.randint(1,5)/100
        b = random.randint(1,4)/100
        c = random.randint(1,9)/10 + random.randint(1,9) + 0.09
        OutQs[0].append(f'Work out £{a} + £{b}')
        OutQs[0].append(f'Work out £{c} + £{b}')
        OutQs[0].append(f'Work out £{a+10} - £{c}')
        OutQs[1].append(f'£{a+b}')
        OutQs[1].append(f'£{c+b}')
        OutQs[1].append(f'£{a+10-c}')
        NumOfQs -= 3
    
    while NumOfQs > 0:
        a = random.randint(1,1000)/100
        b = random.randint(1,int(a*100))/100
        Question = random.choice([[f"Work out £{a} + £{b}",f"£{a+b}"],[f"Work out £{a} - £{b}",f"£{a-b}"]])
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs

#Value of the __ in __
def QA_PKValueOf(NumOfQs, bRamp):
    OutQs = [[],[]]

    if NumOfQs >= 3 and bRamp == True:
        Nums = [1,2,3,4,5,6,7,8,9]
        a = Nums.pop(random.randint(0,len(Nums)-1))
        b = random.choice([Nums.pop(random.randint(0,len(Nums)-1)),0])
        c = Nums.pop(random.randint(0,len(Nums)-1))
        d = random.choice([Nums.pop(random.randint(0,len(Nums)-1)),0])
        OutQs[0].append(f'What is the value of {c} in {a},{b}{c}{d}?')
        OutQs[0].append(f'What is the value of {c} in {a}.{b}{c}{d}?')
        OutQs[0].append(f'What is the value of {c} in {a}{b}{c},{d}00?')
        OutQs[1].append(f'{c}0 (as a number or in words), or {c} tens')
        OutQs[1].append(f'0.0{c} or {c} hundredths')
        OutQs[1].append(f'{c}000 or {c} thousand')
        NumOfQs -= 3
    
    while NumOfQs > 0:
        Nums = [1,2,3,4,5,6,7,8,9]
        a = Nums.pop(random.randint(0,len(Nums)-1))
        b = random.choice([Nums.pop(random.randint(0,len(Nums)-1)),0])
        c = Nums.pop(random.randint(0,len(Nums)-1))
        d = random.choice([Nums.pop(random.randint(0,len(Nums)-1)),0])
        Value = int(str(f"{a}{b}{c}{d}"))
        Factor = random.choice([1,10,100,1000,0.1,0.01])
        Value = Value*Factor
        OutQs[0].append(f'What is the value of {c} in {Value}?')
        OutQs[1].append(f"{c*10*Factor}, as a number or in words.")
        NumOfQs -= 1
    
    return OutQs

#Times by 2.
def QA_Times2(NumOfQs, bRamp):
    OutQs = [[],[]]
    a = 2
    if NumOfQs >= 3 and bRamp == True:
        
        b = 2*random.randint(2,10)
        c = a*b
        OutQs[0].append(f'{int(b/2)} × {a} =')
        OutQs[0].append(f'{b} × {a} =')
        OutQs[0].append(f'{int(b/2)} × {a*2} =')
        OutQs[1].append(f'{int(c/2)}')
        OutQs[1].append(f'{c}')
        OutQs[1].append(f'{c}')
        NumOfQs -= 3
    
    while NumOfQs > 0:
        
        b = random.randint(1,15)*random.choice([1,random.randint(1,5),5,random.choice([10,10,10,100]),random.randint(11,15)])
        Question = MakeMultQuestion(a,b)
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs

#Round to nearest whole number, 10, or 100
def QA_RoundWhole(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:
        a = random.randint(1,99) + 0.4
        OutQs[0].append(f'Round {a} to the nearest whole number.')
        OutQs[0].append(f'Round {a+0.1} to the nearest whole number.')
        OutQs[0].append(f'Round {a+0.42} to the nearest whole number.')
        OutQs[1].append(math.floor(a))
        OutQs[1].append(math.ceil(a))
        OutQs[1].append(math.ceil(a))
        NumOfQs -= 3
    
    while NumOfQs > 0:
        if random.choice([True,False,False]): #round decimal
            a = random.randint(0,random.choice([20,100,1000])) + random.randint(1,9)/10
            OutQs[0].append(f'Round {a} to the nearest whole number.')
            OutQs[1].append(round(a+0.001))
        elif random.choice([True,False]): #round ten
            a = random.randint(1,100)*10 + random.randint(1,9) + random.choice([0,0,random.randint(1,9)/10])
            OutQs[0].append(f'Round {a} to the nearest ten.')
            OutQs[1].append(round((a/10)+0.001)*10)
        else: #round hundred
            a = random.randint(1,random.choice([10,100]))*100 + random.randint(1,99) + random.choice([0,0,random.randint(1,9)/10])
            OutQs[0].append(f'Round {a} to the nearest hundred.')
            OutQs[1].append(round((a/100)+0.001)*100)
        NumOfQs -= 1
        
    return OutQs
        

def QA_TimesHalf(NumOfQs, bRamp):
    OutQs = [[],[]]
    a = 0.5
    if NumOfQs >= 3 and bRamp == True:
        
        b = 2*random.randint(2,10)
        c = int(a*b)
        OutQs[0].append(f'{b} × {a} =')
        OutQs[0].append(f'{(b*2)} × {a} =')
        OutQs[0].append(f'{int(b/2)} ÷ {a} =')
        OutQs[1].append(f'{c}')
        OutQs[1].append(f'{b}')
        OutQs[1].append(f'{b}')
        NumOfQs -= 3
    
    while NumOfQs > 0:
        
        b = random.randint(1,15)*random.choice([1,random.randint(1,5),5,random.choice([10,10,10,100]),random.randint(11,15)])
        Question = MakeMultQuestion(a,b)
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs

#Find the nth multiple of..
def QA_FindMultiplesOf(NumOfQs, bRamp):
    OutQs = [[],[]]
    if NumOfQs >= 3 and bRamp == True:
        
        a = random.randint(1,25)
        b = a*2
        OutQs[0].append(f'What is the 2nd multiple of {b}?')
        OutQs[0].append(f'Find the 4th multiple of {a}.')
        OutQs[0].append(f'{a*8} is which multiple of {a}?')
        OutQs[1].append(b*2)
        OutQs[1].append(a*4)
        OutQs[1].append('The 8th multiple.')
        NumOfQs -= 3
    
    while NumOfQs > 0:
        a = random.randint(1,10)*random.choice([1,1,2,5,10])
        if random.choice([True,False,False]):
            multiple = random.choice([['first',1],['second',2],['third',3],['fourth',4]])
        else:
            num = random.randint(5,20)
            multiple = [f'{num}th',num]
        Question = f"{random.choice(['What is the ','Find the '])}{multiple[0]} multiple of {a}"
        if Question[0] == 'W':
            Question = Question + '?'
        else:
            Question = Question + '.'
        OutQs[0].append(Question)
        OutQs[1].append(multiple[1]*a)
        NumOfQs -= 1
    
    return OutQs
        
#Round to nearest 1 dp or 2 dp
def QA_RoundDecimal(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:
        a = (random.randint(1,9) + 0.4)/10
        OutQs[0].append(f'Round {a} to the nearest decimal place.')
        OutQs[0].append(f'Round {a+0.01} to the nearest one decimal place.')
        OutQs[0].append(f'Round {a+0.007} to the nearest two decimal places.')
        OutQs[1].append(math.floor(a*10)/10)
        OutQs[1].append(math.ceil(a*10)/10)
        OutQs[1].append(math.ceil((a+0.007)*100)/100)
        NumOfQs -= 3
    
    while NumOfQs > 0:
        if random.choice([True,False]):
            a = random.choice([random.randint(1,99)/100,random.randint(1,999)/1000,random.randint(1,999)/100])
            if len(str(a))==3:
                a += random.randint(1,9)/100
            OutQs[0].append(f'Round {a} to the nearest{random.choice([" "," one "])}decimal place.')
            OutQs[1].append(round(a*10 + 0.1)/10)
        else:
            a = random.choice([random.randint(1,999)/1000,random.randint(1,9999)/1000,random.randint(1,99999)/1000])
            OutQs[0].append(f'Round {a} to the nearest two decimal places.')
            OutQs[1].append(round(a*100 + 0.1)/100)
        NumOfQs -= 1
        
    return OutQs

#Multiply by 5, 10, 15, 25
def QA_TimesMult5(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:
        a = random.choice([random.randint(6,9),random.randint(11,12)])
        OutQs[0].append(f'{a} × 5 =')
        OutQs[0].append(f'{a} × 10 =')
        OutQs[0].append(f'{a} × 15 =')
        OutQs[1].append(a*5)
        OutQs[1].append(a*10)
        OutQs[1].append(a*15)
        NumOfQs -= 3
    
    while NumOfQs > 0:
        a = random.choice([random.randint(0,24),random.randint(0,15),random.randint(2,12)])
        if a <= 12:
            b = random.choice([5,10,15,15,25,25])
        else:
            b = random.choice([5,5,5,10,10,15,25])
        Question = MakeMultQuestion(a,b)
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs

#Find the missing number in the sequence eg. 2 .. 5, 6.5, 8
def QA_MissingNumberInSequence(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:
        start = random.randint(1,30)
        step = random.choice([2,3,5])
        OutQs[0].append(f'Find the missing number in the addition sequence: {start}, {start+step}, __, {start+step*3}, {start+step*4}')
        OutQs[0].append(f'Find the missing number: {start/10}, {(start+step)/10}, {(start+step*2)/10}, __, {(start+step*4)/10}')
        OutQs[0].append(f'What is the missing number? {start}, __, {start - step*0.2}, {start-step*0.3}, {start-step*0.4}')
        OutQs[1].append(start+step*2)
        OutQs[1].append((start+step*3)/10)
        OutQs[1].append(start-step*0.1)
        NumOfQs -= 3
    
    while NumOfQs > 0:
        if random.choice([True,False]):
            #Whole number, simple sequence
            start = random.randint(0,100)
            step = random.choice([0.5,2,2.5,3,random.randint(3,12),random.randint(3,20)])*random.choice([1,1,-1])
            sequence = [start-step*3,start-step*2,start-step,start,'__',start+step*2,start+step*3,start+step*4,start+step*5]
        else:
            #Large or small sequence
            factor = random.choice([0.1,0.1,10,100])
            start = random.randint(10,99)*factor
            step = random.choice([random.randint(1,9),random.randint(1,15),random.randint(1,99)])*factor*random.choice([1,1,-1])
            sequence = [start-step*3,start-step*2,start-step,start,'__',start+step*2,start+step*3,start+step*4,start+step*5]
        while len(sequence)>5:
            if random.choice([True,False]):
                for i in range(random.randint(1,len(sequence)-5)):
                    sequence.pop(0)
            else:
                for i in range(random.randint(1,len(sequence)-5)):
                    sequence.pop(-1)            
        sentstart = random.choice(['Find the missing term in the sequence:',
                                   'What is the missing term in the sequence:', 
                                   'Find the missing term in',
                                   'Work out the missing term in'])
        OutQs[0].append(f'{sentstart} {sequence}')
        OutQs[1].append(start+step)
        NumOfQs -= 1
    
    return OutQs

#Column addition and subtraction
def QA_ColumnAddSub(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:
        a = random.randint(1,5)*100 + random.randint(1,5)*10 + random.randint(1,5)
        b = random.randint(1,4)*100 + random.randint(1,4)*10 + random.randint(1,4)
        c = random.randint(1,4)*100 + random.randint(1,4)*10 + 9
        d = random.randint(6,9)*10 + 9
        #COLADD
        OutQs[0].append(f'    {a}\n+ {b}')
        OutQs[0].append(f'    {a}\n+ {c}')
        OutQs[0].append(f'    {b}\n- {d}')
        OutQs[1].append(a+b)
        OutQs[1].append(a+c)
        OutQs[1].append(b-d)
        NumOfQs -= 3
    
    while NumOfQs > 0:
        a = random.randint(100,9999)
        b = random.choice([random.randint(10,99),random.randint(100,9999)])
        if random.choice([True,False,False]):
            Question = [f'    {a}\n +{b}',a+b]
        elif random.choice([True,False]):
            if a<b:
                c = a
                a = b
                b = c
            Question = [f'    {a}\n- {b}',a-b]
        else:
            Question = MakeAddQuestion(a,b)
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs

#Multiply by 2-15
def QA_TimesTo15(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:
        a = random.randint(2,6)
        b = random.randint(7,15)
        c = a*b
        OutQs[0].append(f'{a} × {b} =')
        OutQs[0].append(f'{c} ÷ {b} =')
        OutQs[0].append(f'{c} ÷ {a} =')
        OutQs[1].append(c)
        OutQs[1].append(a)
        OutQs[1].append(b)
        NumOfQs -= 3
    
    while NumOfQs > 0:
        a = random.choice([random.randint(0,15),random.randint(6,15)])
        b = random.choice([random.randint(0,15),random.randint(6,15)])
        Question = MakeMultQuestion(a,b)
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs

#Square numbers
def QA_SquareNumbers(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:
        a = random.randint(2,12)
        b = random.randint(1,3)
        OutQs[0].append(f'{a} × {a} =')
        OutQs[0].append(f'{a}^2 =')
        OutQs[0].append(f'({a} + {b})^2 =')
        OutQs[1].append(a**2)
        OutQs[1].append(a**2)
        OutQs[1].append((a+b)**2)
        NumOfQs -= 3
    
    while NumOfQs > 0:
        a = random.randint(2,15)
        if random.choice([True,False,False]):
            Question = [f'{a}^2 =',a**2]        
        elif random.choice([True,False]):
            b = random.randint(2,15)
            Question = [f'{a}^2 + {b}^2 =',(a**2 + b**2)]
        else:
            Question = [f'__^2 = {a**2}?',a]
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs
#Double digit long multiplication (up to 20)
def QA_LongMultTo20(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:
        a = random.randint(11,20)
        b = random.randint(11,18)
        OutQs[0].append(f'    {a}\n×  {b}')
        OutQs[0].append(f'    {a}\n×  {b+1}')
        OutQs[0].append(f'    {a}\n×  {b+2}')
        #OutQs[0].append(f'COLMULT{a}×{b}')
        #OutQs[0].append(f'COLMULT{a}×{b+1}')
        #OutQs[0].append(f'COLMULT{a}×{b+2}')
        OutQs[1].append(a*b)
        OutQs[1].append(a*(b+1))
        OutQs[1].append(a*(b+2))
        NumOfQs -= 3
    
    while NumOfQs > 0:
        a = random.randint(11,20)
        b = random.randint(11,20)
        Question = [f'    {a}\n×  {b}',a*b]
        #Question = [f'COLMULT{a}×{b}',a*b]
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs

#Cube numbers
def QA_CubeNumbers(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:
        a = random.randint(3,5)
        OutQs[0].append(f'{a}^2 =')
        OutQs[0].append(f'{a} × {a} × {a} =')
        OutQs[0].append(f'{a}^3 =')
        OutQs[1].append(a**2)
        OutQs[1].append(a**3)
        OutQs[1].append(a**3)
        NumOfQs -= 3
    
    while NumOfQs > 0:
        a = random.choice([random.randint(2,6),random.randint(0,12)])
        if random.choice([True,False,False]):
            Question = [f'{a}^3 =',a**3]        
        elif random.choice([True,False]):
            b = random.randint(2,15)
            Question = [f'{a}^3 + {b}^2 =',(a**3 + b**2)]
        else:
            Question = [f'__^3 = {a**3}?',a]
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs

#List the factors of..
def QA_ListFactors(NumOfQs, bRamp):
    OutQs = [[],[]]
    primes = [2,3,5,7,11,13,17,19,23,29,31,37,41]
    
    if NumOfQs >= 3 and bRamp == True:
        a = 2
        b = primes[1:].pop(random.randint(0,2))
        c = primes[1:].pop(random.randint(0,2))
        OutQs[0].append(f'List the factors of {a*b}.')
        OutQs[0].append(f'List the factors of {a*b*2}.')
        OutQs[0].append(f'How many factors are in {a*b*c}?')
        OutQs[1].append(f'1, {a}, {b}, {a*b}')
        OutQs[1].append(f'1, {a}, {b}, {a*2}, {b*2}, {a*b*2}')
        OutQs[1].append(8)
        NumOfQs -= 3
        primes = [2,3,5,7,11,13,17,19,23,29,31,37,41]
    
    while NumOfQs > 0:
        a = primes[random.randint(0,3)]
        b = primes[random.randint(0,3)]
        c = primes[random.randint(0,3)]
        d = primes[random.randint(0,len(primes)-1)]
        if random.choice([True,False]):
            #List the factors of ..
            abcfacs = sorted(set([a,b,c,a*b*c,a*b,a*c,b*c]))
            facstring = "1, "
            for fac in abcfacs:
                facstring = facstring + str(abcfacs.pop(0)) + ", "
            facstring = facstring[:-2]
            OutQs[0].append(f'List the factors of {a*b*c}.')
            OutQs[1].append(facstring)
            
        elif random.choice([True,False]):
            #is (random 1-20) a factor of abd?
            randnum = random.choice([random.randint(2,19),primes[random.randint(0,3)]])
            ans = "No"
            if (a*b*d)%randnum == 0:
                ans = "Yes"
            OutQs[0].append(f'Is {randnum} a factor of {a*b*d}?')
            OutQs[1].append(ans)
        else:
            e = random.randint(2,20)
            f = random.randint(2,12)
            if e==f:
                f += 1
            OutQs[0].append(f'What is the smallest number that has the factors {e} and {f}?')
            OutQs[1].append(np.lcm(e,f))
        NumOfQs -= 1
    
    return OutQs

#Prime number q's
def QA_PrimeNumbers(NumOfQs, bRamp):
    OutQs = [[],[]]
    primes = [   2,    3,    5,    7,   11,   13,   17,   19,   23,   29,   31,
         37,   41,   43,   47,   53,   59,   61,   67,   71,   73,   79,
         83,   89,   97,  101,  103,  107,  109,  113,  127,  131,  137,
        139,  149,  151,  157,  163,  167,  173,  179,  181,  191,  193,
        197,  199,  211,  223,  227,  229,  233,  239,  241,  251,  257,
        263,  269,  271,  277,  281,  283,  293,  307,  311,  313,  317,
        331,  337,  347,  349,  353,  359,  367,  373,  379,  383,  389,
        397,  401,  409,  419,  421,  431,  433,  439,  443,  449,  457,
        461,  463,  467,  479,  487,  491,  499,  503,  509,  521,  523,
        541,  547,  557,  563,  569,  571,  577,  587,  593,  599,  601,
        607,  613,  617,  619,  631,  641,  643,  647,  653,  659,  661,
        673,  677,  683,  691,  701,  709,  719,  727,  733,  739,  743,
        751,  757,  761,  769,  773,  787,  797,  809,  811,  821,  823,
        827,  829,  839,  853,  857,  859,  863,  877,  881,  883,  887,
        907,  911,  919,  929,  937,  941,  947,  953,  967,  971,  977,
        983,  991,  997, 1009]
    
    if NumOfQs >= 3 and bRamp == True:
        a = random.randint(8,14)
        b = primes[a]
        c = primes[a]+2
        while c in primes:
            c += 2
        OutQs[0].append(f'Is {b} a prime number?')
        OutQs[0].append(f'Is {c} a prime number?')
        OutQs[0].append(f'{primes[a+2]} is prime. Find the next prime number after {primes[a+2]}')
        OutQs[1].append('Yes')
        OutQs[1].append('No')
        OutQs[1].append(primes[a+3])
        NumOfQs -= 3
    
    while NumOfQs > 0:
        if random.choice([True,False,False]):
            a = primes[random.randint(5,len(primes)-1)]
            Question = [f"Is {a} a prime number?","Yes"]
        elif random.choice([True,False]):
            a = primes[random.randint(5,len(primes)-4)] + 2
            while a in primes:
                a += 2
            Question = [f"Is {a} a prime number?","No"]
        else:
            n = random.randint(1,10)
            Question = [f"What is the {n}^(th) prime number?",primes[n-1]]
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs



#Long multiplication/division with a 2-digit and 3-digit
def QA_LongMult2dig3dig(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:
        a = random.randint(11,99)
        b = random.randint(21,198)*5
        OutQs[0].append(f'    {a}\n×  {b}')
        OutQs[0].append(f'    {a}\n×  {b+1}')
        OutQs[0].append(f'    {a}\n×  {b+2}')
        OutQs[1].append(a*b)
        OutQs[1].append(a*(b+1))
        OutQs[1].append(a*(b+2))
        NumOfQs -= 3
    
    while NumOfQs > 0:
        a = random.randint(11,99)
        b = random.randint(101,999)
        Question = [f'    {a}\n×  {b}',a*b]
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs

#Multiply by powers of 10
def QA_TimesPowerOfTen(NumOfQs, bRamp):
    OutQs = [[],[]]
    
    if NumOfQs >= 3 and bRamp == True:    
        a = random.randint(1,99) * random.choice([1,0.1])
        OutQs[0].append(f'{a} × 10 =')
        OutQs[0].append(f'{a} × 100 =')
        OutQs[0].append(f'{a} ÷ 10 =')
        OutQs[1].append(10*a)
        OutQs[1].append(100*a)
        OutQs[1].append(0.1*a)
        NumOfQs -= 3
    
    while NumOfQs > 0:
        a = random.randint(1,99) * random.choice([1,0.1,10])
        b = random.choice([10,100,1000,0.1,0.01])
        Question = MakeMultQuestion(a,b)
        OutQs[0].append(Question[0])
        OutQs[1].append(Question[1])
        NumOfQs -= 1
    
    return OutQs

In [46]:
#Challenge Puzzles: Easy
#Note: Puzzles store questions a little differently, as [qstring1,qstring2,qstring3 (etc),astring]

#[Puzzle name, weight] where weight is 3 (common), 2 (uncommon), 1 (rare)
Puzzles = [["AnimalNoises","OddsEvens","IncreasingMoney","UseNumbersOnce",
           "ItMakesZero","FollowTheChain","Hailstone","Binary","WhichTwoNumbers"],[2,1,3,3,1,2,1,2,3]]

#Returns [[Questions],[Answers]]
def RandomPuzzle(HowMany):
    ProbabilityTotal = sum(Puzzles[1])
    OutPuzzles = [[],[]]
    while HowMany > 0:
        Choice = random.randint(1,ProbabilityTotal)
        ArrayCounter = 0
        ArraySum = Puzzles[1][0]
        while Choice>ArraySum:
            ArrayCounter += 1
            ArraySum += Puzzles[1][ArrayCounter]
        HowMany -= 1
        Puzzle = globals()["Puzz_"+Puzzles[0][ArrayCounter]]()
        #OutPuzzles[0] = OutPuzzles[0] + Puzzle[:-1]
        OutPuzzles[0].append(Puzzle[:-1])
        OutPuzzles[1].append(Puzzle[-1])
    return OutPuzzles

#Random Animals (Uncommon puzzle)
Animals = [["frog","croaks"],["lion","roars"],["dog","barks"],
           ["cat","meows"],["lizard's tail","wags"],["cow","moos"],["sheep","baas"]]
def Puzz_AnimalNoises():
    
    Animal = random.choice(Animals)
    Frequency = random.randint(3,27)
    Minutes = random.randint(2,9) 
    return [f"A {Animal[0]} {Animal[1]} every {Frequency} seconds.", f"In {Minutes} minutes, how many times will it {Animal[1][:-1]}?",
           math.floor(Minutes*60/Frequency)]

#Odds & Evens (rare puzzle)
def Puzz_OddsEvens():
    
    Str1 = "Odd × odd = odd. An example of this is: 3 x 5 = 15."
    Str2 = "Odd x even = even. An example of this is: 7 x 8 = 56."
    List1 = random.choice([["What is even x odd?","even"], ["What is even x even?","even"]])
    List2 = random.choice([["What is even + odd?","odd"], ["What is even + even?","even"]])
    Str3 = "For both questions that you have answered, give an example to show you are right."
    num = random.randint(3,5)
    if num%2 == 0:
        ans = "even"
    else:
        ans = "odd"
    num2 = random.randint(10,20)
    if num2%2 == 0:
        ans2 = "even"
    else:
        ans2 = "odd"
    List5 = [f"If you were to add together {num} odd numbers, would the answer be odd or even?",ans]
    List6 = [f"If you were to add together {num2} odd numbers, would the answer be odd or even?",ans2]
    Anslist = f"{List1[1]}, {List2[1]}, {List5[1]}, {List6[1]}"
    return[random.choice([Str1,Str2]),
          List1[0],
          List2[0],
          Str3,
          List5[0],
          List6[0],
          Anslist]

#This is part of the below puzzle.
def FindSum(pence,days):
    OutSum = 0
    while days > 0:
        OutSum += days*pence
        days -= 1
    return OutSum/100
    
#Increasing money puzzle. Common.
def Puzz_IncreasingMoney():
    pence = random.randint(1,5)
    days = random.randint(10,30)
    return[f"Each day, you get {pence} pence more than the day before.",
          f"On day 1, you get {pence} pence.",
          f"On day 2, you get {pence*2} pence, so in total you have {pence*3} pence.",
          f"How much will you have on day {days}?",
          f"£{FindSum(pence,days)}"]

#Common.
def Puzz_UseNumbersOnce():
    numbs = [1,2,3,4,5,6,7,8,9,10,11,12]
    if random.choice([True,False,False]):
        numbs = list(np.array(numbs)*0.1)
    a = numbs.pop(random.randint(0,len(numbs)-1))
    b = numbs.pop(random.randint(0,len(numbs)-1))
    ab = a*b
    c = numbs.pop(random.randint(0,len(numbs)-1))
    d = numbs.pop(random.randint(0,len(numbs)-1))
    cd = c*d
    e = numbs.pop(random.randint(0,len(numbs)-1))
    f = numbs.pop(random.randint(0,len(numbs)-1))
    epf = e+f
    numbs = [a,ab,d,cd,e,f]
    random.shuffle(numbs)
    return[f"Using the numbers below once only, make the calculations correct.",
          f"{numbs[0]}, {numbs[1]}, {numbs[2]}, {numbs[3]}, {numbs[4]}, {numbs[5]}",
          f"___ + ___ = {epf}",
           f"___ × {b} = ___",
           f"___ = ___ ÷ {c}",
          f"{e} + {f} = {epf}, {a} × {b} = {ab}, {d} = {cd} ÷ {c}"]

#Rare
def Puzz_ItMakesZero():
    a = random.randint(2,8)
    b = random.randint(2,8)
    c = random.randint(2,8)
    d = random.randint(2,8)
    e = random.randint(2,6)*random.randint(2,6)
    return[f"a) Work out the following: ({a} × {b} × {c} × {d} × {e}) - ({a*c} × {b*e} × {d})",
          "b) When you figure out how this works, can you make a question like this of your own?",
          "0"]

#Uncommon
def Puzz_FollowTheChain():
    Names = ["Jess","Stuart","Jemma","Cindy","Daniel","Simon","Khamani","Oliver","Amy","Phebe","Alexis","Ben","Cara"]
    Namea = Names.pop(random.randint(0,len(Names)-1))
    Nameb = Names.pop(random.randint(0,len(Names)-1))
    Namec = Names.pop(random.randint(0,len(Names)-1))
    Named = Names.pop(random.randint(0,len(Names)-1))
    Things = ["donuts","marbles","coins","candles","cards","coupons"]
    Thing = Things.pop(random.randint(0,len(Things)-1))
    a = random.randint(2,5)
    b = random.randint(2,10)
    c = random.randint(2,20)
    return[f"{Namea} has {a} {Thing}.",
          f"{Nameb} has {b} times as many {Thing} as {Namea}.",
          f"{Namec} has {c} more {Thing} than {Nameb}.",
          f"{Named} and {Namea} together have as many {Thing} as {Namec}.",
          f"How many {Thing} do they all have altogether?",
          a+(2*a*b)+c+((a*b)+c-a)]

#Rare
def Puzz_Hailstone():
    a = random.randint(1,10)
    return ["In the Hailstone Sequence, you start with a number and use the following two rules:",
           "If the number is odd, you multiply it by 3 and add 1.",
           "If the number is even, you divide it by 2.",
            f"Continue the sequence, starting with the number {a} until you notice a pattern. What is that pattern?",
           "Try out a few more numbers to be sure of the pattern.",
           "Answer may vary based on the number used, but some mathematicians think the pattern will always end up at 1."]
    
#Uncommon
def Puzz_Binary():
    a = random.randint(10,20)
    binarynums = [1, 10, 11, 100, 101, 110, 111, 1000,1001,1010,1011,1100,1101,1110,1111,10000,10001,10010,10011,10100]
    return ["In Binary, only 0 and 1 can be used. the numbers 1-8 are as follows:",
            "1, 10, 11, 100, 101, 110, 111, 1000",
           f"Can you continue this? What would the number {a} be?",
           f"{binarynums[a-1]}"]

#Common
def Puzz_WhichTwoNumbers():
    primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]
    a = primes[random.randint(1,11)]
    b = primes[random.randint(5,len(primes)-1)]
    return[f"Other than 1 and itself, which two numbers multiply to make {a*b}?",
          f"{a} and {b}"]

    

In [6]:
#document = Document()
#styles = document.styles
#for style in styles:
#    print("'%s' -- %s" % (style.name, style.type))
print(ITopics)

['Filler',
 'PKValueOf',
 'PKMoney',
 'PKTimesTablesTo6',
 'PK24hrTime',
 'PKHoursToMins',
 'Times2',
 'TimesHalf',
 'TimesMult5',
 'TimesTo15',
 'TimesPowerOfTen',
 'LongMultTo20',
 'LongMult2dig3dig',
 'ColumnAddSub',
 'ListFactors',
 'PrimeNumbers',
 'RoundWhole',
 'RoundDecimal',
 'FindMultiplesOf',
 'MissingNumberInSequence',
 'SquareNumbers',
 'CubeNumbers']